<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/07_document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 単語埋め込みを使った文書分類
* 今回は、fastTextのような学習済みの単語埋め込みは使わない。
* 単語埋め込み自体の学習も、ネットワークの重みの学習と同時におこなう。
* IMDbデータの準備も、PyTorchのtorchtextモジュールを使っておこなう。
* ネットワークへの入力は、単語埋め込みを、単語の出現順どおりに並べた列にする。
* そして、前向き計算のなかではじめて、単語埋め込みの平均をとることにする。
* 参考資料
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

## 07-00 Google Colabのランタイムのタイプを変更する
* Google ColabのランタイムのタイプをGPUに変更しておこう。
 * 上のメニューの「ランタイム」→「ランタイムのタイプを変更」→「ハードウェア　アクセラレータ」から「GPU」を選択

## 07-01 torchtextを使ってIMDbデータを読み込む
* https://torchtext.readthedocs.io/en/latest/datasets.html

### 実験の再現性確保のための設定など
* torch.backends.cudnn.deterministicをTrueにするのは、こうしないと、GPU上での計算が毎回同じ値を与えないため。

In [1]:
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### torchtextのフィールド
* TEXTフィールドと、LABELフィールドという２種類のFieldオブジェクトのインスタンスを作る。
* TEXTフィールドは、テキストの前処理の仕方を決めておくのに使う。
 * tokenizerは、デフォルトでは単にstring型のsplitメソッドを適用するだけになる。これは高速だが、tokenizationとしては雑。
* LABELフィールドは、ラベルの前処理に使う。

In [2]:
TEXT = data.Field(tokenize="spacy")
LABEL = data.LabelField()

### IMDbデータセットを前処理しつつ読み込む
* TEXTフィールドでspaCyのtokenizationを使うように設定したので、少し時間がかかる。
 * string型splitメソッドでtokenizeすると時間はあまりかからない。

In [3]:
train_dev_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

### 最初の文書を見てみる
（ちなみに、`vars`関数は、モジュール、クラス、インスタンス、あるいはそれ以外の`__dict__`属性を持つオブジェクトの、`__dict__`属性を辞書として返す組み込み関数。）

In [4]:
print(vars(train_dev_data.examples[0]))

{'text': ['The', 'vigilante', 'has', 'long', 'held', 'a', 'fascination', 'for', 'audiences', ',', 'inasmuch', 'as', 'it', 'evokes', 'a', 'sense', 'of', 'swift', ',', 'sure', 'justice', ';', 'good', 'triumphs', 'over', 'evil', 'and', 'the', 'bad', 'guy', 'gets', 'his', 'deserts', '.', 'It', 'is', ',', 'in', 'fact', ',', 'one', 'of', 'the', 'things', 'that', 'has', 'made', 'the', 'character', 'of', 'Dirty', 'Harry', 'Callahan', '(', 'as', 'played', 'by', 'Clint', 'Eastwood', ')', 'so', 'popular', '.', 'He', 'carries', 'a', 'badge', 'and', 'works', 'within', 'the', 'law', ',', 'but', 'at', 'heart', ',', 'Harry', 'is', 'a', 'vigilante', ',', 'meting', 'out', 'justice', '`', 'his', "'", 'way', ',', 'which', 'often', 'puts', 'him', 'in', 'conflict', 'with', 'his', 'own', 'superiors', ',', 'as', 'well', 'as', 'the', 'criminals', 'he', "'s", 'pursuing', '.', 'But', 'it', "'s", 'what', 'draws', 'the', 'audience', ';', 'anyone', 'who', "'s", 'ever', 'been', 'bogged', 'down', 'in', 'bureaucratic'

In [5]:
print(train_dev_data.examples[0].text)

['The', 'vigilante', 'has', 'long', 'held', 'a', 'fascination', 'for', 'audiences', ',', 'inasmuch', 'as', 'it', 'evokes', 'a', 'sense', 'of', 'swift', ',', 'sure', 'justice', ';', 'good', 'triumphs', 'over', 'evil', 'and', 'the', 'bad', 'guy', 'gets', 'his', 'deserts', '.', 'It', 'is', ',', 'in', 'fact', ',', 'one', 'of', 'the', 'things', 'that', 'has', 'made', 'the', 'character', 'of', 'Dirty', 'Harry', 'Callahan', '(', 'as', 'played', 'by', 'Clint', 'Eastwood', ')', 'so', 'popular', '.', 'He', 'carries', 'a', 'badge', 'and', 'works', 'within', 'the', 'law', ',', 'but', 'at', 'heart', ',', 'Harry', 'is', 'a', 'vigilante', ',', 'meting', 'out', 'justice', '`', 'his', "'", 'way', ',', 'which', 'often', 'puts', 'him', 'in', 'conflict', 'with', 'his', 'own', 'superiors', ',', 'as', 'well', 'as', 'the', 'criminals', 'he', "'s", 'pursuing', '.', 'But', 'it', "'s", 'what', 'draws', 'the', 'audience', ';', 'anyone', 'who', "'s", 'ever', 'been', 'bogged', 'down', 'in', 'bureaucratic', 'nonsen

In [6]:
print(train_dev_data.examples[0].label)

pos


### テストセット以外の部分を訓練データと検証データに分ける

In [7]:
train_data, dev_data = train_dev_data.split(split_ratio=0.8, random_state = random.seed(SEED))

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of development examples: {len(dev_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of development examples: 5000
Number of testing examples: 25000


### データセットのラベルを作る
* TEXTラベルのほうでは、最大語彙サイズを指定する。

In [9]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

なぜ語彙サイズが25,000ではなく25,002なのか、については少し下の説明を参照。

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


### 出現頻度順で上位２０単語を見てみる

In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 231547), (',', 220008), ('.', 189104), ('and', 125224), ('a', 124944), ('of', 115034), ('to', 107158), ('is', 87000), ('in', 70317), ('I', 62117), ('it', 61401), ('that', 56179), ('"', 50540), ("'s", 49238), ('this', 48511), ('-', 42040), ('/><br', 40630), ('was', 39971), ('as', 34721), ('with', 34233)]


### 単語ID順に最初の１０単語を見てみる
* IDのうち、0と1は、未知語とパディング用の単語という特殊な単語に割り振られている。
 * パディングとは、長さが不揃いの複数の文書を同じミニバッチにまとめるとき、すべての文書の長さを無理やりそろえるため、文書末尾に特殊な単語（元々の語彙にない、人工的に用意した単語）を追加すること。

In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


### ラベルのほうのIDを確認する
* こちらはnegとposに対応する２つのIDしかない。

In [13]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fea7b6062f0>, {'neg': 0, 'pos': 1})


### ミニバッチを取り出すためのiteratorを作る
* ミニバッチのサイズを指定する。
 * ミニバッチのサイズは、性能を出すためにチューニングすべきハイパーパラメータのひとつ。

In [14]:
BATCH_SIZE = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, dev_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, dev_data, test_data),
    batch_size=BATCH_SIZE,
    device=device)

### 試しにテストセットのiteratorを回してミニバッチをすべて取得して個数を数えてみる

In [15]:
i = 0
for batch in test_iterator:
  i += 1
  continue
print(f'We have {i} mini-batches in test set.')

We have 250 mini-batches in test set.


### ミニバッチの中身を見てみる
* 上記のループを抜けたあとには、変数batchにはテストセットの最後のミニバッチが代入されている。
* そこで、この最後のミニバッチのshapeを確認する。

### ミニバッチのshapeを確認する
* ミニバッチの形は、[ミニバッチに含まれる最長文書の文書長, ミニバッチのサイズ]になっていることに注意！
 * ミニバッチのサイズが最初に来ているのではない！
* [ミニバッチのサイズ, ミニバッチに含まれる最長文書の文書長]という形にしたいときは、テキストのfieldを作るときにに以下のようにする。

__`TEXT = data.Field(tokenize="spacy", batch_first=True)`__

In [16]:
batch.text.shape

torch.Size([2640, 100])

このミニバッチに含まれる文書のうち、最初のものの単語ID列と、先頭100個のIDを単語に戻したものを表示させてみる。

In [17]:
print(batch.text[:, 0])
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:100, 0]]))

tensor([157,  15,   6,  ...,   7, 315,   4], device='cuda:0')
There 's a sign on The Lost Highway that <unk> /><br <unk> SPOILERS <unk> /><br <unk> you already knew that , did n't <unk> /><br />Since there 's a great deal of people that apparently did not get the point of this movie , I 'd like to contribute my interpretation of why the plot makes perfect sense . As others have pointed out , one single viewing of this movie is not sufficient . If you have the DVD of <unk> , you can " cheat " by looking at David Lynch 's " Top 10 <unk> to <unk>


最後の文書の末尾は「1」で埋められていることが分かる。

In [18]:
print(batch.text[:, BATCH_SIZE-1])

tensor([  14,   25, 2791,  ...,    1,    1,    1], device='cuda:0')


ミニバッチに含まれる文書の長さを調べると、文書が文書長の降順に並べられていることが分かる。

In [19]:
(batch.text != 1).sum(0)

tensor([2640, 2537, 2534, 1915, 1635, 1364, 1324, 1290, 1287, 1284, 1279, 1274,
        1264, 1255, 1253, 1241, 1241, 1232, 1229, 1226, 1225, 1221, 1213, 1212,
        1211, 1210, 1206, 1205, 1198, 1196, 1195, 1193, 1193, 1193, 1191, 1190,
        1190, 1187, 1185, 1183, 1182, 1181, 1181, 1181, 1181, 1179, 1179, 1177,
        1176, 1175, 1175, 1175, 1173, 1173, 1173, 1172, 1172, 1172, 1171, 1170,
        1169, 1169, 1168, 1167, 1166, 1166, 1164, 1164, 1163, 1163, 1162, 1161,
        1159, 1158, 1158, 1157, 1157, 1155, 1155, 1153, 1152, 1152, 1151, 1150,
        1149, 1148, 1147, 1146, 1145, 1144, 1144, 1144, 1144, 1144, 1144, 1143,
        1143, 1142, 1142, 1142], device='cuda:0')

## 07-02 MLPによる文書分類の準備
* 今回は、ごく簡単なMLPで文書分類をする。
* 文書中の全単語トークンの埋め込みベクトルの平均を、MLPの入力とする。
 * 当然、語順の情報は使われない。

### 定数の設定
* 単語埋め込みベクトルの次元数は100にする。

In [20]:
INPUT_DIM = len(TEXT.vocab)
NUM_CLASS = len(LABEL.vocab)
EMBED_DIM = 100
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

### モデルを定義する前にPyTorchの単語埋め込みがどんなものかを見てみる

以下のように、語彙サイズと埋め込みの次元数を指定しつつ、torch.nn.Embeddingのインスタンスを作ればよい。

In [21]:
embed = nn.Embedding(INPUT_DIM, EMBED_DIM, padding_idx=PAD_IDX)

パディング用の単語の埋め込みはゼロベクトルになる。

In [22]:
print(embed(torch.tensor([[2,3,4],[0,2,1]])))

tensor([[[ 1.1721, -0.4372, -0.4053,  0.7086,  0.9533, -0.0130, -0.1301,
          -0.0877, -0.0673,  0.2467, -0.9392, -1.0448,  1.2783,  0.4190,
          -0.5073, -0.6062, -1.0532,  1.8386, -0.1095, -0.3316,  0.9008,
           0.4840, -1.3237,  0.7869,  1.3818, -0.0694, -0.7612,  0.2416,
          -0.5878, -1.1506,  1.0164,  0.1234,  1.1311, -0.0858, -0.0597,
           0.3553, -1.4355,  0.0727,  0.1053, -1.0311,  1.3113, -0.0360,
           0.2118, -0.0086,  1.8576,  2.1321, -0.5056, -0.7988, -1.0944,
          -1.0197, -0.5399,  1.2117, -0.8632,  1.3337,  0.0771, -0.0522,
           0.2386,  0.1411, -1.3354, -2.9340,  0.1141, -1.2072, -0.3008,
           0.1427, -1.3027, -0.4919, -2.1429,  0.9488, -0.5684, -0.0646,
           0.6647, -2.7836,  1.1366,  0.9089,  0.9494,  0.0266, -0.9221,
           0.7034, -0.3659, -0.1965, -0.9207,  0.3154, -0.0217,  0.3441,
           0.2271, -0.4597, -0.6183,  0.2461, -0.4055, -0.8368,  1.2277,
          -0.4297, -2.2121, -0.3780,  0.9838, -1.08

### モデルの定義
* MLP（多層パーセプトロン）だが、入り口に単語埋め込み層が挿入されている。

In [23]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.fc1.weight.data.uniform_(-initrange, initrange)
    self.fc1.bias.data.zero_()
    self.fc2.weight.data.uniform_(-initrange, initrange)
    self.fc2.bias.data.zero_()
    self.fc3.weight.data.uniform_(-initrange, initrange)
    self.fc3.bias.data.zero_()

  def forward(self, text):
    x = self.embed(text)
    x = x.mean(0) 
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

### モデルを作る
* モデル（のインスタンス）をGPUに移動させている点に注意。

In [24]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX).to(device)

### 損失関数とoptimizerとschedulerを作る

In [25]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

### 訓練用の関数

In [26]:
def train(data_iterator, model, optimizer, scheduler, criterion):

  model.train()

  train_loss = 0
  train_acc = 0
  for batch in data_iterator:
    optimizer.zero_grad()
    text, cls = batch.text, batch.label
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).sum().item()

  scheduler.step()

  data_len = len(data_iterator.dataset)
  return train_loss / data_len, train_acc / data_len

### 評価用の関数

In [27]:
def test(data_iterator, model, criterion):

  model.eval()

  loss = 0
  acc = 0
  for batch in data_iterator:
    text, cls = batch.text, batch.label
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      loss += loss.item()
      acc += (output.argmax(1) == cls).sum().item()

  data_len = len(data_iterator.dataset)
  return loss / data_len, acc / data_len

## 07-03 分類器の訓練と開発セットでの評価

In [28]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  dev_loss, dev_acc = test(dev_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

Epoch: 1  | time in 0 minutes, 5 seconds
	Loss: 0.0044(train)	|	Acc: 79.4%(train)
	Loss: 0.0001(dev)	|	Acc: 88.5%(dev)
Epoch: 2  | time in 0 minutes, 5 seconds
	Loss: 0.0017(train)	|	Acc: 93.6%(train)
	Loss: 0.0001(dev)	|	Acc: 89.4%(dev)
Epoch: 3  | time in 0 minutes, 5 seconds
	Loss: 0.0008(train)	|	Acc: 97.2%(train)
	Loss: 0.0001(dev)	|	Acc: 88.0%(dev)
Epoch: 4  | time in 0 minutes, 5 seconds
	Loss: 0.0004(train)	|	Acc: 98.9%(train)
	Loss: 0.0001(dev)	|	Acc: 88.1%(dev)
Epoch: 5  | time in 0 minutes, 5 seconds
	Loss: 0.0002(train)	|	Acc: 99.4%(train)
	Loss: 0.0002(dev)	|	Acc: 87.7%(dev)
Epoch: 6  | time in 0 minutes, 5 seconds
	Loss: 0.0001(train)	|	Acc: 99.6%(train)
	Loss: 0.0001(dev)	|	Acc: 87.5%(dev)
Epoch: 7  | time in 0 minutes, 5 seconds
	Loss: 0.0000(train)	|	Acc: 99.9%(train)
	Loss: 0.0002(dev)	|	Acc: 87.3%(dev)
Epoch: 8  | time in 0 minutes, 5 seconds
	Loss: 0.0000(train)	|	Acc: 99.9%(train)
	Loss: 0.0002(dev)	|	Acc: 87.3%(dev)
Epoch: 9  | time in 0 minutes, 5 seconds
	Loss: 

## 07-04 再検討
* 訓練データ上での分類精度が100%になってしまっている。明らかにオーバーフィッティング。

### ドロップアウトを使う
* モデルのインスタンスを作るときにdropoutの確率を引数pで指定できるようにする。

In [29]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx, p=0.0):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.dropout = nn.Dropout(p=p)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.fc1.weight.data.uniform_(-initrange, initrange)
    self.fc1.bias.data.zero_()
    self.fc2.weight.data.uniform_(-initrange, initrange)
    self.fc2.bias.data.zero_()
    self.fc3.weight.data.uniform_(-initrange, initrange)
    self.fc3.bias.data.zero_()

  def forward(self, text):
    x = self.dropout(self.embed(text))
    x = x.mean(0)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [30]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [31]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  dev_loss, dev_acc = test(dev_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

Epoch: 1  | time in 0 minutes, 6 seconds
	Loss: 0.0051(train)	|	Acc: 74.7%(train)
	Loss: 0.0001(dev)	|	Acc: 86.8%(dev)
Epoch: 2  | time in 0 minutes, 6 seconds
	Loss: 0.0024(train)	|	Acc: 90.3%(train)
	Loss: 0.0001(dev)	|	Acc: 88.8%(dev)
Epoch: 3  | time in 0 minutes, 6 seconds
	Loss: 0.0017(train)	|	Acc: 93.7%(train)
	Loss: 0.0001(dev)	|	Acc: 88.6%(dev)
Epoch: 4  | time in 0 minutes, 6 seconds
	Loss: 0.0013(train)	|	Acc: 95.3%(train)
	Loss: 0.0001(dev)	|	Acc: 88.9%(dev)
Epoch: 5  | time in 0 minutes, 6 seconds
	Loss: 0.0009(train)	|	Acc: 96.8%(train)
	Loss: 0.0001(dev)	|	Acc: 88.7%(dev)
Epoch: 6  | time in 0 minutes, 6 seconds
	Loss: 0.0007(train)	|	Acc: 97.3%(train)
	Loss: 0.0001(dev)	|	Acc: 88.5%(dev)
Epoch: 7  | time in 0 minutes, 6 seconds
	Loss: 0.0006(train)	|	Acc: 97.7%(train)
	Loss: 0.0001(dev)	|	Acc: 88.3%(dev)
Epoch: 8  | time in 0 minutes, 6 seconds
	Loss: 0.0005(train)	|	Acc: 98.2%(train)
	Loss: 0.0001(dev)	|	Acc: 88.3%(dev)
Epoch: 9  | time in 0 minutes, 6 seconds
	Loss: 

### L２正則化を使う
* optimizerのweight_decayパラメータを0より大きな値にする。

In [32]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [33]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  dev_loss, dev_acc = test(dev_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

Epoch: 1  | time in 0 minutes, 6 seconds
	Loss: 0.0067(train)	|	Acc: 60.9%(train)
	Loss: 0.0002(dev)	|	Acc: 69.2%(dev)
Epoch: 2  | time in 0 minutes, 6 seconds
	Loss: 0.0052(train)	|	Acc: 74.4%(train)
	Loss: 0.0001(dev)	|	Acc: 80.1%(dev)
Epoch: 3  | time in 0 minutes, 6 seconds
	Loss: 0.0045(train)	|	Acc: 80.2%(train)
	Loss: 0.0002(dev)	|	Acc: 80.1%(dev)
Epoch: 4  | time in 0 minutes, 6 seconds
	Loss: 0.0040(train)	|	Acc: 83.0%(train)
	Loss: 0.0001(dev)	|	Acc: 85.4%(dev)
Epoch: 5  | time in 0 minutes, 6 seconds
	Loss: 0.0039(train)	|	Acc: 83.3%(train)
	Loss: 0.0001(dev)	|	Acc: 84.2%(dev)
Epoch: 6  | time in 0 minutes, 6 seconds
	Loss: 0.0036(train)	|	Acc: 85.2%(train)
	Loss: 0.0001(dev)	|	Acc: 84.7%(dev)
Epoch: 7  | time in 0 minutes, 6 seconds
	Loss: 0.0035(train)	|	Acc: 85.8%(train)
	Loss: 0.0001(dev)	|	Acc: 87.1%(dev)
Epoch: 8  | time in 0 minutes, 6 seconds
	Loss: 0.0034(train)	|	Acc: 86.4%(train)
	Loss: 0.0001(dev)	|	Acc: 87.1%(dev)
Epoch: 9  | time in 0 minutes, 6 seconds
	Loss: 

### early stopping
* dev setでのaccuracyが4回連続で最高値を下回ったら訓練を終えることにする。
* early stoppingの実現については、PyTorch Lightningを使う手もある。
 * https://pytorch-lightning.readthedocs.io/en/latest/early_stopping.html

In [34]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [35]:
patience = 4
early_stop_count = 0
best_dev_acc = 0.0
dev_acc_threshold = 0.87

N_EPOCHS = 50 # エポック数を増やしておく
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  dev_loss, dev_acc = test(dev_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

  # early stopping
  if best_dev_acc <= dev_acc:
    best_dev_acc = dev_acc
    early_stop_count = 0
  else:
    early_stop_count += 1
    if early_stop_count == patience:
      break

Epoch: 1  | time in 0 minutes, 6 seconds
	Loss: 0.0065(train)	|	Acc: 62.6%(train)
	Loss: 0.0002(dev)	|	Acc: 73.1%(dev)
Epoch: 2  | time in 0 minutes, 6 seconds
	Loss: 0.0050(train)	|	Acc: 76.6%(train)
	Loss: 0.0001(dev)	|	Acc: 82.8%(dev)
Epoch: 3  | time in 0 minutes, 6 seconds
	Loss: 0.0045(train)	|	Acc: 79.1%(train)
	Loss: 0.0001(dev)	|	Acc: 84.0%(dev)
Epoch: 4  | time in 0 minutes, 6 seconds
	Loss: 0.0040(train)	|	Acc: 82.2%(train)
	Loss: 0.0001(dev)	|	Acc: 83.3%(dev)
Epoch: 5  | time in 0 minutes, 6 seconds
	Loss: 0.0038(train)	|	Acc: 84.2%(train)
	Loss: 0.0001(dev)	|	Acc: 86.4%(dev)
Epoch: 6  | time in 0 minutes, 6 seconds
	Loss: 0.0036(train)	|	Acc: 84.9%(train)
	Loss: 0.0001(dev)	|	Acc: 86.7%(dev)
Epoch: 7  | time in 0 minutes, 6 seconds
	Loss: 0.0034(train)	|	Acc: 86.0%(train)
	Loss: 0.0001(dev)	|	Acc: 87.0%(dev)
Epoch: 8  | time in 0 minutes, 6 seconds
	Loss: 0.0034(train)	|	Acc: 85.9%(train)
	Loss: 0.0001(dev)	|	Acc: 86.4%(dev)
Epoch: 9  | time in 0 minutes, 6 seconds
	Loss: 

## 07-05 テストセット上で評価
* 見つけ出したベストな設定を使って、テストセット上での最終的な評価をおこなう。

In [37]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_iterator, model, criterion)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0000(test)	|	Acc: 88.2%(test)
